In [ ]:
# default_exp cleaner

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [ ]:
df = pd.read_csv("Buldozers.csv", low_memory=False)
df.head().T

,0,1,2,3,4
Unnamed: 0,0,1,2,3,4
SalesID,1139246,1139248,1139249,1139251,1139253
SalePrice,66000,57000,10000,38500,11000
MachineID,999089,117657,434808,1026470,1057373
ModelID,3157,77,7009,332,17311
datasource,121,121,121,121,121
auctioneerID,3,3,3,3,3
YearMade,2004,1996,2001,2001,2007
MachineHoursCurrentMeter,68,4640,2838,3486,722
UsageBand,Low,Low,High,High,Medium


# Handle Dates

In [ ]:
def add_datepart(df, fldnames, drop=True, time=False, errors="raise"):	
    """add_datepart converts a column of df from a datetime64 to many columns containing
    the information from the date. This applies changes inplace.
    Parameters:
    -----------
    df: A pandas data frame. df gain several new columns.
    fldname: A string or list of strings that is the name of the date column you wish to expand.
        If it is not a datetime64 series, it will be converted to one with pd.to_datetime.
    drop: If true then the original date column will be removed.
    time: If true time features: Hour, Minute, Second will be added.
    Examples:
    ---------
    >>> df = pd.DataFrame({ 'A' : pd.to_datetime(['3/11/2000', '3/12/2000', '3/13/2000'], infer_datetime_format=False) })
    >>> df
        A
    0   2000-03-11
    1   2000-03-12
    2   2000-03-13
    >>> add_datepart(df, 'A')
    >>> df
        AYear AMonth AWeek ADay ADayofweek ADayofyear AIs_month_end AIs_month_start AIs_quarter_end AIs_quarter_start AIs_year_end AIs_year_start AElapsed
    0   2000  3      10    11   5          71         False         False           False           False             False        False          952732800
    1   2000  3      10    12   6          72         False         False           False           False             False        False          952819200
    2   2000  3      11    13   0          73         False         False           False           False             False        False          952905600
    >>>df2 = pd.DataFrame({'start_date' : pd.to_datetime(['3/11/2000','3/13/2000','3/15/2000']),
                            'end_date':pd.to_datetime(['3/17/2000','3/18/2000','4/1/2000'],infer_datetime_format=True)})
    >>>df2
        start_date	end_date    
    0	2000-03-11	2000-03-17
    1	2000-03-13	2000-03-18
    2	2000-03-15	2000-04-01
    >>>add_datepart(df2,['start_date','end_date'])
    >>>df2
    	start_Year	start_Month	start_Week	start_Day	start_Dayofweek	start_Dayofyear	start_Is_month_end	start_Is_month_start	start_Is_quarter_end	start_Is_quarter_start	start_Is_year_end	start_Is_year_start	start_Elapsed	end_Year	end_Month	end_Week	end_Day	end_Dayofweek	end_Dayofyear	end_Is_month_end	end_Is_month_start	end_Is_quarter_end	end_Is_quarter_start	end_Is_year_end	end_Is_year_start	end_Elapsed
    0	2000	    3	        10	        11	        5	            71	            False	            False	                False	                False	                False	            False	            952732800	    2000	    3	        11	        17	    4	            77	            False	            False	            False	            False	                False	        False	            953251200
    1	2000	    3	        11	        13	        0	            73	            False	            False	                False	                False               	False           	False           	952905600     	2000       	3	        11      	18  	5           	78          	False	            False           	False           	False               	False          	False           	953337600
    2	2000	    3	        11	        15	        2           	75          	False           	False               	False               	False               	False               False           	953078400      	2000    	4          	13      	1   	5           	92          	False           	True            	False           	True                	False          	False           	954547200
    """
    if isinstance(fldnames,str): 
        fldnames = [fldnames]
        
    for fldname in fldnames:
        fld = df[fldname]
        fld_dtype = fld.dtype
        if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
            fld_dtype = np.datetime64

        if not np.issubdtype(fld_dtype, np.datetime64):
            df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True, errors=errors)
        targ_pre = re.sub('[Dd]ate$', '', fldname)
        attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
                'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
        if time: 
            attr = attr + ['Hour', 'Minute', 'Second']
        for n in attr:
            if n=='Week':
                df[targ_pre + n] = getattr(fld.dt.isocalendar(), n.lower())
            else:
                df[targ_pre + n] = getattr(fld.dt, n.lower())
        df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
        if drop: 
            df.drop(fldname, axis=1, inplace=True)

def is_date(x): return np.issubdtype(x.dtype, np.datetime64)

In [ ]:
df_temp = df[['saledate','Ripper']].copy()
add_datepart(df_temp, 'saledate')
assert len(df_temp.columns)==14
assert 'saledate' not in df_temp.columns

In [ ]:
df_temp = df[['saledate','Ripper']].copy()
add_datepart(df_temp, 'saledate', time=True)
assert len(df_temp.columns)==17
assert 'saledate' not in df_temp.columns

# Fill Missing

In [ ]:
def is_numeric(var):
    numerics = [float,np.float64, int, np.int64, pd.Int64Dtype]
    if type(var) is pd.Series:
        return True if var.dtype in numerics else False
    else:
        return True if type(var) in numerics else False

In [ ]:
def fill_missing_numeric(df:pd.DataFrame, col:str, na_dict={None:None}):
    """
    Fills missing missing values with the median or value specified in na_dict if it's used
    
    Parameters:
    df: dataframe
    col: name of column to fill missing values
    na_dict: (optional) dict with name of the column as key specific value that is used to fill missing values
    
    returning:
    alters the dataframe, adding a 'col_na' column indicating which parameters have been filled
    return a dictionary containing the value that was used to fill missing.
    
    """
    if is_numeric(df[col]):
        if df[col].isnull().sum():
            df[col + '_na'] = df[col].isnull()
            filler = na_dict[col] if col in na_dict else df[col].median()
            print(df[col].median(), filler)
            df[col] = df[col].fillna(filler)
            na_dict[col] = filler
    return na_dict

In [ ]:
df_test = pd.DataFrame({'A':[1,2,np.nan]})
d = fill_missing_numeric(df_test, col='A')

1.5 1.5


In [ ]:
assert d['A'] == 1.5
assert len(df_test.columns)==2
assert 'A_na' in df_test.columns

In [ ]:
df_test = pd.DataFrame({'A':[1,2,2,np.nan]}, dtype=pd.Int64Dtype)
d = fill_missing_numeric(df_test, col='A')

2.0 1.5


In [ ]:
df_test['A'].median()

2.0

In [ ]:
df_test

,A,A_na
0,1.0,False
1,2.0,False
2,2.0,False
3,1.5,True


In [ ]:
a = df_test['A'].median().astype(pd.Int64Dtype)

In [ ]:
pd.Int64Dtype

pandas.core.arrays.integer.Int64Dtype

In [ ]:
df_test

,A
0,1
1,2
2,2
3,NaN


In [ ]:
df_test.median()

A    2.0
dtype: float64

In [ ]:
df_test

,A,A_na
0,1.0,False
1,2.0,False
2,2.0,False
3,1.5,True


In [ ]:
#Fill missing categorical?
#typer av kategorier: int, få värden, borde<- finns väl inte efter som int blir float när NAN kommer in i bilden

In [ ]:
# kolla om unika värden är under xx
# Om string < 6, one-hot encoda dessa
# om int el float < 20, sortera och skapa kategorier
# om float, kolla om man behöver ta log eller exp på parametrarna?